# Librerias

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import glob
import os

In [ ]:
import folium
from folium import plugins

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Configuraciones

In [ ]:
# NOMBRE DEL PROYECTO
PROYECTO = 'Def2_Planet'

# BANDAS Y VARIABLE RESPUESTA
opticalBands = ['R', 'G', 'B', 'NIR','R2', 'G2', 'B2', 'NIR2']
#opticalBands = ['Band_1', 'Band_2', 'Band_3', 'Band_4']
BANDS = opticalBands
RESPONSE = 'landcover'
FEATURES = BANDS + [RESPONSE]

# ESPECIFICACIONES DE LOS DATOS DE ENTRADA DEL MODELO DE SEGMENTACIÓN
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]

# Datos de entrada

## Visualizar imágenes y vectores

In [ ]:
periodo = 'T3'

if periodo == 'T1':
  imagen1 = ee.Image('users/mazarazaa/201512_PLDef').select('b1', 'b2', 'b3', 'b4').rename('B', 'G', 'R', 'NIR').unmask(0)
  imagen2 = ee.Image('users/mazarazaa/201612_PLDef').select('b1', 'b2', 'b3', 'b4').rename('B2', 'G2', 'R2', 'NIR2').unmask(0)
  imagen = imagen1.addBands(imagen2)
  vector = ee.FeatureCollection('users/mazarazaa/Cmb_201612_201512_def')
  rgbParams = {'bands': ['R', 'G', 'B'],
             'min': 20,
             'max': 3500}
elif periodo == 'T2':
  imagen1 = ee.Image('users/mazarazaa/201612_PLDef').select('b1', 'b2', 'b3', 'b4').rename('B', 'G', 'R', 'NIR').unmask(0)
  imagen2 = ee.Image('users/mazarazaa/201906_PLDef').select('b1', 'b2', 'b3', 'b4').rename('B2', 'G2', 'R2', 'NIR2').unmask(0)
  imagen = imagen1.addBands(imagen2)
  vector = ee.FeatureCollection('users/mazarazaa/Cmb_201906_201612_def')
  rgbParams = {'bands': ['R', 'G', 'B'],
             'min': 20,
             'max': 3500}
elif periodo == 'T3':
  imagen1 = ee.Image('users/mazarazaa/201512_PLDef').select('b1', 'b2', 'b3', 'b4').rename('B', 'G', 'R', 'NIR').unmask(0)
  imagen2 = ee.Image('users/mazarazaa/201906_PLDef').select('b1', 'b2', 'b3', 'b4').rename('B2', 'G2', 'R2', 'NIR2').unmask(0)
  imagen = imagen1.addBands(imagen2)
  vector = ee.FeatureCollection('users/mazarazaa/Cmb_201906_201512_def')
  rgbParams = {'bands': ['R', 'G', 'B'],
             'min': 50,
             'max': 2500}

map = folium.Map(location=[2.543, -74.267], zoom_start=12)
map.add_ee_layer(imagen, rgbParams, 'Imagen')
map.add_ee_layer(vector, {'palette': 'ebe834'}, 'Vector')

map.add_child(folium.LayerControl())
map

## Rasterizar vectores

In [ ]:
def set_landcover(ft):
  return ft.set('landcover', 1)

# Definir valor de la clase del vector 
vector_clasificado = vector.map(set_landcover)

# Crear una imagen base con todos los valores iguales a cero
blankimg = ee.Image.constant(0)

# Convertir vectores a imagen
vector_imagen = vector_clasificado.reduceToImage(['landcover'], ee.Reducer.first())

# Convertir pixels de la imagen base donde se sobrepone los de la imagen y renombrar como landcover
etiqueta_imagen = blankimg.where(vector_imagen, vector_imagen).rename('landcover')

vizParams = {'bands': 'landcover', 'min':0, 'max': 1}

map = folium.Map(location = [2.543, -74.267], zoom_start=12)
map.add_ee_layer(etiqueta_imagen,  vizParams, 'Imagen de etiquetas')
map.add_child(folium.LayerControl())
map


#### **Partición**

In [ ]:
## generar los bloques usando la funcion de require('users/gena/packages:grid') - Solo disponible en GEE JS
# var imagen = ee.Image('projects/ee-multitemporal-gee4geo/assets/construcciones/Antioquia_2017')
# var vector = ee.FeatureCollection('users/maikolzaraza07/construcciones_2010_ant')

# var crs = imagen.projection().crs();
# var region = vector.geometry().bounds()

# var g = require('users/gena/packages:grid')

# var proj = crs
# var dx = 2500
# var dy = 2500
# var marginx = -200, marginy = -200
# var particion = g.generateGridForGeometry(region, dx, dy, marginx, marginy, proj)

particion = ee.FeatureCollection('users/mazarazaa/Planet_particionDef')

withRandom = particion.randomColumn('random',1);
split = 0.3;
trainPolys = withRandom.filter(ee.Filter.gte('random', split));
restPolys = withRandom.filter(ee.Filter.lt('random', split));
print(trainPolys.size().getInfo())

withRandom2 = restPolys.randomColumn('random',1);
split = 0.7;
validationPolys = withRandom2.filter(ee.Filter.lt('random', split));
evalPolys = withRandom2.filter(ee.Filter.gte('random', split));

print(validationPolys.size().getInfo())
print(evalPolys.size().getInfo())

In [ ]:
map = folium.Map(location=[2.543, -74.267], zoom_start=12)
map.add_ee_layer(imagen, rgbParams, 'Color')
map.add_ee_layer(trainPolys, {'palette': '2c7fb8'}, 'Train blocks')
map.add_ee_layer(validationPolys, {'palette': '7fcdbb'}, 'Validation blocks')
map.add_ee_layer(evalPolys, {'palette': 'edf8b1'}, 'Test blocks')
map.add_child(folium.LayerControl())
map

# Generación de muestras

In [ ]:
def buildGeometryPoint(pnt):
    return pnt.setGeometry(ee.Geometry.Point([pnt.get("longitude"), pnt.get("latitude")]))

In [ ]:
#crear carpeta de salida
out_dir = os.path.join('/content/gdrive/My Drive',PROYECTO)

os.makedirs(out_dir, exist_ok=True)

In [ ]:
target_imagen = imagen
resolucion = target_imagen.projection().nominalScale()

featureStack = ee.Image.cat([
  target_imagen.select(BANDS),
  etiqueta_imagen.select(RESPONSE)
]).float()

In [ ]:
TRAIN_BASE = 'training_' + periodo
VAL_BASE = 'validation_' + periodo
EVAL_BASE = 'eval_' + periodo

list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

arrays = featureStack.neighborhoodToArray(kernel)

# Convert the feature collections to lists for iteration.
trainingPolysList = trainPolys.toList(trainPolys.size())
validationPolysList = validationPolys.toList(validationPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())

# These numbers determined experimentally.
n = 5 # Number of shards in each polygon.
N = 50 # Total sample size in each polygon.

In [ ]:
# Export all the training data (in many pieces), with one task 
# per geometry.
geomSample_loc_train = ee.FeatureCollection([])
for g in range(trainingPolysList.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    pntCol = (featureStack.select('landcover').rename('class').toInt().addBands(ee.Image.pixelLonLat())
        .stratifiedSample(
            region=ee.Feature(trainingPolysList.get(g)).geometry(),
            numPoints=N / n,
            classBand='class',
            scale=resolucion,
            seed=i,
            dropNulls=True,
            tileScale=8
        ).map(buildGeometryPoint)
    )

    sample = arrays.sampleRegions(
              collection=pntCol,
              scale=resolucion,
              tileScale=8
            )

    geomSample = geomSample.merge(sample)
    geomSample_loc_train = geomSample_loc_train.merge(pntCol)

  desc = TRAIN_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection=geomSample,
    description=desc,
    fileNamePrefix = desc,
    folder=PROYECTO,
    fileFormat='TFRecord',
    selectors=BANDS + [RESPONSE]
  )

  task.start()

geomSample_loc_val = ee.FeatureCollection([])
for g in range(validationPolysList.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    pntCol = (featureStack.select('landcover').rename('class').toInt().addBands(ee.Image.pixelLonLat())
        .stratifiedSample(
            region=ee.Feature(validationPolysList.get(g)).geometry(),
            numPoints=N / n,
            classBand='class',
            scale=resolucion,
            seed=i,
            dropNulls=True,
            tileScale=8
        ).map(buildGeometryPoint)
    )

    sample = arrays.sampleRegions(
              collection=pntCol,
              scale=resolucion,
              tileScale=8
            )

    geomSample = geomSample.merge(sample)
    geomSample_loc_val = geomSample_loc_val.merge(pntCol)

  desc = VAL_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection=geomSample,
    description=desc,
    fileNamePrefix = desc,
    folder=PROYECTO,
    fileFormat='TFRecord',
    selectors=BANDS + [RESPONSE]
  )

  task.start()

geomSample_loc_eval = ee.FeatureCollection([])
for g in range(evalPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    pntCol = (featureStack.select('landcover').rename('class').toInt().addBands(ee.Image.pixelLonLat())
        .stratifiedSample(
            region=ee.Feature(evalPolysList.get(g)).geometry(),
            numPoints=N / n,
            classBand='class',
            scale=resolucion,
            seed=i,
            dropNulls=True,
            tileScale=8
        ).map(buildGeometryPoint)
    )

    sample = arrays.sampleRegions(
              collection=pntCol,
              scale=resolucion,
              tileScale=8
            )

    geomSample = geomSample.merge(sample)
    geomSample_loc_eval = geomSample_loc_eval.merge(pntCol)

  desc = EVAL_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection=geomSample,
    description=desc,
    fileNamePrefix = desc,
    folder=PROYECTO,
    fileFormat='TFRecord',
    selectors=BANDS + [RESPONSE]
  )

  task.start()

In [ ]:
import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(5000)

In [ ]:
TRAIN_SIZE = geomSample_loc_train.size().getInfo()
VAL_SIZE = geomSample_loc_val.size().getInfo()
EVAL_SIZE = geomSample_loc_eval.size().getInfo()
print(f'TRAIN_SIZE: {TRAIN_SIZE}')
print(f'VAL_SIZE: {VAL_SIZE}')
print(f'EVAL_SIZE: {EVAL_SIZE}')

In [ ]:
def buff(ft):
  return ft.buffer(resolucion.multiply(KERNEL_SIZE/2)).geometry().bounds()

VizParams = {'bands': 'landcover', 'min':0, 'max': 1}

# Create a folium map object.
my_map = folium.Map(location=[2.543, -74.267], zoom_start=12, height=500)

my_map.add_ee_layer(etiqueta_imagen,  VizParams, 'Imagen vectores')

geomSample_loc_train = ee.FeatureCollection('users/mazarazaa/Def2_Planet_T1_muestras_training-N50_n5').merge(ee.FeatureCollection('users/mazarazaa/Def2_Planet_T2_muestras_training-N50_n5'))
geomSample_loc_val = ee.FeatureCollection('users/mazarazaa/Def2_Planet_T1_muestras_validation-N50_n5').merge(ee.FeatureCollection('users/mazarazaa/Def2_Planet_T2_muestras_validation-N50_n5'))
geomSample_loc_eval = ee.FeatureCollection('users/mazarazaa/Def2_Planet_T1_muestras_evaluation-N50_n5').merge(ee.FeatureCollection('users/mazarazaa/Def2_Planet_T2_muestras_evaluation-N50_n5'))

# Display ee.FeatureCollection
my_map.add_ee_layer(geomSample_loc_train.filter(ee.Filter.eq('class',0)).map(buff), {'palette': 'd334eb'}, 'Puntos Muestra Training - No Cambio')
my_map.add_ee_layer(geomSample_loc_train.filter(ee.Filter.eq('class',1)).map(buff), {'palette': 'ebe834'}, 'Puntos Muestra Training - Cambio')
my_map.add_ee_layer(geomSample_loc_val.filter(ee.Filter.eq('class',0)).map(buff), {'palette': '3f0f46'}, 'Puntos Muestra Validation - No Cambio')
my_map.add_ee_layer(geomSample_loc_val.filter(ee.Filter.eq('class',1)).map(buff), {'palette': 'ebe7ec'}, 'Puntos Muestra Validation - Cambio')
my_map.add_ee_layer(geomSample_loc_eval.filter(ee.Filter.eq('class',0)).map(buff), {'palette': '2f2e2f'}, 'Puntos Muestra Evaluation - No Cambio')
my_map.add_ee_layer(geomSample_loc_eval.filter(ee.Filter.eq('class',1)).map(buff), {'palette': '816285'}, 'Puntos Muestra Evaluation - Cambio')
my_map.add_ee_layer(trainPolys, {'palette': '2c7fb8'}, 'Train blocks')
my_map.add_ee_layer(validationPolys, {'palette': '7fcdbb'}, 'Validation blocks')
my_map.add_ee_layer(evalPolys, {'palette': 'edf8b1'}, 'Test blocks')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

## Exportar los features (bloques y muestras) como assets

In [ ]:
TRAIN_SIZE = geomSample_loc_train.size().getInfo()
VAL_SIZE = geomSample_loc_val.size().getInfo()
EVAL_SIZE = geomSample_loc_eval.size().getInfo()
print(f'TRAIN_SIZE: {TRAIN_SIZE}')
print(f'VAL_SIZE: {VAL_SIZE}')
print(f'EVAL_SIZE: {EVAL_SIZE}')

## Bloques

In [ ]:
outdir = 'users/mazarazaa'
particiones = {'training':trainPolys,'validation':validationPolys,'evaluation':evalPolys}

for particion in particiones.keys():
  task = ee.batch.Export.table.toAsset(
    collection=particiones[particion],
    description=particion,
    assetId=os.path.join(outdir,f'{PROYECTO}_bloques_{particion}'))
  
  task.start()

### Muestras

In [ ]:
outdir = 'users/mazarazaa'
particiones = {'training':geomSample_loc_train,'validation':geomSample_loc_val,'evaluation':geomSample_loc_eval}

for particion in particiones.keys():
  task = ee.batch.Export.table.toAsset(
    collection=particiones[particion],
    description=particion,
    assetId=os.path.join(outdir,f'{PROYECTO}_{periodo}_muestras_{particion}-N{N}_n{n}'))
  
  task.start()